In [2]:
!nvcc --version
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc4jupyter

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-0fveh1tz
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-0fveh1tz
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 5741c522547756ac4bb7a16df32106a15efb8a57
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.2.1-py3-none-any.whl size=10741 sha256=1c43b610d84440f376c57bb0b3d20f87e2433fdb6f06eb98d5fd7c81c461a971
  Stored in directory: /tmp/pip-ephem-wheel-cache-rm1zpxb6/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built nvcc4jupyter


In [ ]:
%%cuda
#include <thrust/device_vector.h>
#include <thrust/sort.h>
#include <thrust/unique.h>
#include <thrust/execution_policy.h>
#include <stdio.h>

struct Edge {
    int u, v;
    __host__ __device__
    Edge(int _u = 0, int _v = 0) : u(_u < _v ? _u : _v), v(_u < _v ? _v : _u) {} // Ensure smaller index first

    __host__ __device__
    bool operator<(const Edge& other) const {
        return u == other.u ? v < other.v : u < other.u;
    }

    __host__ __device__
    bool operator==(const Edge& other) const {
        return u == other.u && v == other.v;
    }
};

__global__ void createEdgeList(Edge* edges, int* R, int* C, int num_nodes) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < num_nodes) {
        for (int j = R[idx]; j < R[idx + 1]; j++) {
            edges[j] = Edge(idx, C[j]);
        }
    }
}

int main() {
    // CSR representation of the graph
    int h_R[] = {0, 2, 5, 7, 9};  // Row pointer
    int h_C[] = {1, 2, 0, 2, 3, 0, 3, 2, 3};  // Column indices
    int num_nodes = sizeof(h_R)/sizeof(h_R[0]) - 1;
    int num_edges = sizeof(h_C)/sizeof(h_C[0]);

    // Allocate memory on the device
    thrust::device_vector<int> R(h_R, h_R + num_nodes + 1);
    thrust::device_vector<int> C(h_C, h_C + num_edges);
    thrust::device_vector<Edge> edges(num_edges);

    // Launch kernel to create edge list
    int threadsPerBlock = 256;
    int blocksPerGrid = (num_nodes + threadsPerBlock - 1) / threadsPerBlock;
    createEdgeList<<<blocksPerGrid, threadsPerBlock>>>(thrust::raw_pointer_cast(edges.data()), thrust::raw_pointer_cast(R.data()), thrust::raw_pointer_cast(C.data()), num_nodes);

    // Sort edges
    thrust::sort(thrust::device, edges.begin(), edges.end());

    // Remove duplicates
    auto it = thrust::unique(thrust::device, edges.begin(), edges.end());
    edges.erase(it, edges.end());

    // Copy back and print results
    std::vector<Edge> output_edges(edges.size());
    thrust::copy(edges.begin(), edges.end(), output_edges.begin());
    printf("Unique edges:\n");
    for (const auto& edge : output_edges) {
        printf("{%d, %d}\n", edge.u, edge.v);
    }

    return 0;
}


Unique edges:
{0, 1}
{0, 2}
{1, 2}
{1, 3}
{2, 3}
{3, 3}



In [5]:
%%cuda
#include <iostream>
#include <vector>

// CUDA kernel to remove self-loops from a graph represented in CSR format
__global__ void removeSelfLoops(int *rowPtr, int *colIndices, int numVertices) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;

    if (tid < numVertices) {
        int start = rowPtr[tid];
        int end = rowPtr[tid + 1];

        // Iterate through the column indices of the current row
        for (int i = start; i < end; ++i) {
            // If the column index is equal to the current row, it's a self-loop
            if (colIndices[i] == tid) {
                // Shift elements in colIndices to remove the self-loop
                for (int j = i; j < end - 1; ++j) {
                    colIndices[j] = colIndices[j + 1];
                }
                // Decrement the end position since we removed an element
                --end;
            }
        }
        // Update the row pointer for the next row
        rowPtr[tid + 1] = end;
    }
}

int main() {
    // Example CSR representation of the graph
    std::vector<int> rowPtr = {0, 2, 5, 7, 9};  // Row pointer array
    std::vector<int> colIndices = {1, 2, 1, 3, 0, 2, 3, 1, 3};  // Column indices array

    // Print the original CSR representation
    std::cout << "Original CSR representation:\n";
    std::cout << "Row Pointer Array: ";
    for (int i : rowPtr) {
        std::cout << i << " ";
    }
    std::cout << "\nColumn Indices Array: ";
    for (int i : colIndices) {
        std::cout << i << " ";
    }
    std::cout << std::endl;

    // Allocate memory on the GPU
    int *d_rowPtr, *d_colIndices;
    cudaMalloc(&d_rowPtr, rowPtr.size() * sizeof(int));
    cudaMalloc(&d_colIndices, colIndices.size() * sizeof(int));

    // Copy data from host to device
    cudaMemcpy(d_rowPtr, rowPtr.data(), rowPtr.size() * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_colIndices, colIndices.data(), colIndices.size() * sizeof(int), cudaMemcpyHostToDevice);

    // Calculate number of vertices
    int numVertices = rowPtr.size() - 1;

    // Define block size and grid size
    int blockSize = 256;
    int gridSize = (numVertices + blockSize - 1) / blockSize;

    // Launch the CUDA kernel
    removeSelfLoops<<<gridSize, blockSize>>>(d_rowPtr, d_colIndices, numVertices);

    // Copy the results back to host
    cudaMemcpy(rowPtr.data(), d_rowPtr, rowPtr.size() * sizeof(int), cudaMemcpyDeviceToHost);
    cudaMemcpy(colIndices.data(), d_colIndices, colIndices.size() * sizeof(int), cudaMemcpyDeviceToHost);

    // Print the CSR representation after removing self-loops
    std::cout << "\nCSR representation after removing self-loops:\n";
    std::cout << "Row Pointer Array: ";
    for (int i : rowPtr) {
        std::cout << i << " ";
    }
    std::cout << "\nColumn Indices Array: ";
    for (int i : colIndices) {
        std::cout << i << " ";
    }
    std::cout << std::endl;

    // Free device memory
    cudaFree(d_rowPtr);
    cudaFree(d_colIndices);

    return 0;
}

Original CSR representation:
Row Pointer Array: 0 2 5 7 9 
Column Indices Array: 1 2 1 3 0 2 3 1 3 

CSR representation after removing self-loops:
Row Pointer Array: 0 2 4 6 8 
Column Indices Array: 1 2 3 0 0 3 3 1 3 



In [6]:
%%cuda
#include <iostream>
#include <vector>
#include <cuda_runtime.h>

using namespace std;

// CUDA kernel to remove isolated vertices from a graph in CSR format
__global__ void removeIsolatedVerticesKernel(int* d_rowPtr, int* d_colIndices, int sizeCol, int* d_values, int* d_isolated, int numRows, int* d_visited) {
    int tid = threadIdx.x + blockIdx.x * blockDim.x;

    if (tid < numRows) {
        if (d_rowPtr[tid] < sizeCol) {
            // Vertex tid has neighbors, mark it and its neighbors as not isolated
            d_visited[tid] = 1;
            for (int j = d_rowPtr[tid]; j < d_rowPtr[tid + 1]; ++j) {
                d_visited[d_colIndices[j]] = 1;
            }
        }
    }
}

// Function to remove isolated vertices from a graph in CSR format using CUDA
void removeIsolatedVerticesCUDA(vector<int>& rowPtr, vector<int>& colIndices, vector<int>& values) {
    int numRows = rowPtr.size() - 1;
    int* d_rowPtr;
    int* d_colIndices;
    int* d_values;
    int* d_isolated;
    int* d_visited;
    int* visited = new int[numRows];
    for (int i = 0; i < numRows; ++i) {
        visited[i] = 0; // Initially all vertices are considered isolated
    }

    // Allocate memory on the device
    cudaMalloc(&d_rowPtr, rowPtr.size() * sizeof(int));
    cudaMalloc(&d_colIndices, colIndices.size() * sizeof(int));
    cudaMalloc(&d_values, values.size() * sizeof(int));
    cudaMalloc(&d_isolated, numRows * sizeof(int));
    cudaMalloc(&d_visited, numRows * sizeof(int));

    // Copy data from host to device
    cudaMemcpy(d_rowPtr, rowPtr.data(), rowPtr.size() * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_colIndices, colIndices.data(), colIndices.size() * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_values, values.data(), values.size() * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_visited, visited, numRows * sizeof(int), cudaMemcpyHostToDevice);

    // Call kernel to mark isolated vertices
    int blockSize = 256;
    int numBlocks = (numRows + blockSize - 1) / blockSize;
    removeIsolatedVerticesKernel<<<numBlocks, blockSize>>>(d_rowPtr, d_colIndices, colIndices.size(), d_values, d_isolated, numRows, d_visited);
    cudaDeviceSynchronize();

    // Copy back isolated vector from device to host
    cudaMemcpy(visited, d_visited, numRows * sizeof(int), cudaMemcpyDeviceToHost);

    // Print adjacency list representation
    cout << "Adjacency list representation:" << endl;
    for (int i = 0; i < numRows; ++i) {
        if (visited[i] == 1) {
            cout << i << ": ";
            for (int j = rowPtr[i]; j < rowPtr[i + 1]; ++j) {
                cout << colIndices[j] << " ";
            }
            cout << endl;
        }
    }
    cout << endl;

    // Free device memory
    cudaFree(d_rowPtr);
    cudaFree(d_colIndices);
    cudaFree(d_values);
    cudaFree(d_isolated);
    cudaFree(d_visited);

    delete[] visited;
}

int main() {
    // Example CSR format representation of the graph with an isolated vertex (vertex 4)
    vector<int> rowPtr = {0, 2, 5, 7, 8, 8}; // Note: vertex 4 has no neighbors
    vector<int> colIndices = {1, 3, 0, 2, 1, 0, 3}; // Only vertex 4 has no connections
    vector<int> values = {1, 1, 1, 1, 1, 1, 1};

    // Remove isolated vertices using CUDA
    removeIsolatedVerticesCUDA(rowPtr, colIndices, values);

    return 0;
}

Adjacency list representation:
0: 1 3 
1: 0 2 1 
2: 0 3 
3: 0 


